# SPH - Philip Mocz

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as ani
from scipy.special import gamma

In [2]:
N = 100 # number of particles
M = 2 # star mass
m = M/N # particle mass
R = 0.75 # star radius
h = 0.04 / np.sqrt(N/1000) # smoothing length
dt = .04 # time step
nu = 1 # damping
k = 0.1 # pressure constant
n = 1 # polytropic index
L = 400 # max time steps

In [3]:
def kernel(position):
    
    C_h = 1 / (4*np.pi*h**3)
    q = np.linalg.norm(position) / h
    
    if q >= 2:
        w = 0
    elif q < 2 and q >= 1:
        w = (2-q)**3
    else:
        w = (2-q)**3 - 4*(1-q)**3
    
    return C_h * w

def kernel_gradient(position):
    
    C_h = 3 / (4*np.pi*h**4)
    q = np.linalg.norm(position) / h
    
    if q >= 2:
        w = 0
    elif q < 2 and q >= 1:
        w = (2-q)**2
    else:
        w = (2-q)**2 - 4*(1-q)**2
    
    return C_h * w

In [4]:
def find_density(positions,i):
    total = 0
    for j in range(N):
        r_ij = positions[i,:] - positions[j,:]
        total += kernel(r_ij)
    return m * total
    
def find_pressure(density):
    return k * density**(1 + 1/n)

In [5]:
lambdaFactor = 2*k*(1+n)*np.pi**(-3/(2*n)) * ( (M*gamma(5/2 + n)) / (R**3 * gamma(1+n)) )**(1/n) / R**2

In [11]:
def find_acceleration(positions,velocities,densities,pressures,i):
    
    total = 0
    for j in range(N):
        if i != j:
            r_ij = positions[i,:] - positions[j,:]
            pressure_term = (pressures[i]/densities[i]**2 + pressures[j]/densities[j]**2) * kernel_gradient(r_ij)
            gravity_term = kernel(r_ij) * m**2 / np.linalg.norm(r_ij)**2
            total += pressure_term + gravity_term
            
    return -nu*velocities[i,:] - m*total - lambdaFactor*positions[i,:]

In [7]:
def placement_uniform_sphere():
    """
    make a random distribution of 'N' particles
    inside a sphere.

    returns
    -------
    2D position array

    thanks
    ------
    https://stackoverflow.com/a/5408843
    """

    # set up 1D position arrays in SPC
    U           =   np.random.uniform(0,1,N)
    COSTHETA    =   np.random.uniform(-1,1,N)

    RADIUS  =   R * U**(1/3)
    THETA   =   np.arccos(COSTHETA)
    PHI     =   np.random.uniform(0,2*np.pi,N)

    # set up 2D position array
    X       =   np.zeros( (N,3) )
    # convert SPC to CC
    X[:,0]  =   RADIUS * np.sin(THETA) * np.cos(PHI)
    X[:,1]  =   RADIUS * np.sin(THETA) * np.sin(PHI)
    X[:,2]  =   RADIUS * np.cos(THETA)

    return X

In [12]:
def run():
    
    rmax    =   R*1.2
    
    positions = placement_uniform_sphere()
    velocities = np.zeros_like(positions)
    
    fig =   plt.figure(figsize=[15,15])
    ax  =   fig.gca(projection='3d')
    ax.set_aspect(1)
    ax.plot(positions[:,0],positions[:,1],positions[:,2], 'go')
    
    FFMpegWriter    =   ani.writers['ffmpeg']
    metadata        =   dict(title='SPH Star Formation', artist='Matplotlib')
    writer          =   FFMpegWriter(fps=15, metadata=metadata)
    
    with writer.saving(fig,"SPH.mp4", 100):
        for t in range(100):
   
            ax.clear()

            densities = np.array([ find_density(positions,i) for i in range(N) ])
            pressures = find_pressure(densities)
            accelerations = np.array([ find_acceleration(positions,velocities,densities,pressures,i) for i in range(N) ])

            positions_half = positions + velocities*(dt/2)
            velocities_half = velocities + accelerations*(dt/2)
            densities_half = np.array([ find_density(positions_half,i) for i in range(N) ])
            pressures_half = find_pressure(densities_half)
            accelerations_half = np.array([ find_acceleration(positions_half,velocities_half,densities_half,pressures_half,i) for i in range(N) ])

            positions += velocities_half*dt
            velocities += accelerations_half*dt

            ax.plot(positions[:,0],positions[:,1],positions[:,2], 'go')
            ax.set_aspect(1)
            ax.set_xlim([-rmax,rmax])
            ax.set_ylim([-rmax,rmax])
            ax.set_zlim([-rmax,rmax])
            writer.grab_frame()
    
    plt.close()

run()